## Risk-adjusted mean consumption growth, MC method

In [1]:
%run src/bycv_model.py

In [2]:
by = BYCV()

Here's the spectral radius at default values:

In [3]:
stability_exp_analytic(by)

-0.003154479711489406

In [4]:
f = stability_exponent_mc_factory(by, parallel_flag=False)

In [5]:
%time f(m=1500, n=1000)

CPU times: user 465 ms, sys: 9.12 ms, total: 474 ms
Wall time: 524 ms


-0.0031928646389559638

Once more with parallelization:

In [6]:
f = stability_exponent_mc_factory(by, parallel_flag=True)

In [7]:
%time f(m=1500, n=1000)

CPU times: user 884 ms, sys: 0 ns, total: 884 ms
Wall time: 695 ms


-0.0032532406156597846

Generate the table

In [8]:
n_vals = np.array([250, 500, 750])
m_vals = np.array([1000, 2000, 3000, 4000, 5000])

In [9]:
k = 1000
MC_draws = np.empty(k)

MC_means = np.empty((len(n_vals), len(m_vals)))
MC_stds = np.empty((len(n_vals), len(m_vals)))

for n_i, n in enumerate(n_vals):
    for m_i, m in enumerate(m_vals):
        print(f'Calculating n={n}, m={m}')
        
        for i in range(k):
            MC_draws[i] = f(n=n, m=m)
                
        MC_means[n_i, m_i] = MC_draws.mean()
        MC_stds[n_i, m_i] = MC_draws.std()
        

Calculating n=250, m=1000
Calculating n=250, m=2000
Calculating n=250, m=3000
Calculating n=250, m=4000
Calculating n=250, m=5000
Calculating n=500, m=1000
Calculating n=500, m=2000
Calculating n=500, m=3000
Calculating n=500, m=4000
Calculating n=500, m=5000
Calculating n=750, m=1000
Calculating n=750, m=2000
Calculating n=750, m=3000
Calculating n=750, m=4000
Calculating n=750, m=5000


Convert standard deviations into standard errors for the mean:

In [11]:
m=1500
MC_ses = MC_stds * np.sqrt(1 / m)

In [12]:
def make_table(means, stds):
    means_strings = means.round(7).astype(str)
    n_strings = n_vals.astype(str)

    start_table = r"""
    \begin{table}
    \centering
    \begin{tabular}{llll}
    """

    m_table = ' & m = '.join(m_vals.astype(str))
    m_table = '    & m = ' + m_table + r' \\' + '\n' + r'\hline \hline' '\n'

    end_table = r"""
    \end{tabular}
    \end{table}
    """

    row_string = ''
    for row in range(len(n_strings)):
        temp_means = ' & '.join(means_strings[row, :])

        x = ['{:f}'.format(item) for item in stds[row, :]]    
        temp_stds = '(' + ') & ('.join(x) + ')'
        row_string += f'n = {n_strings[row]} & ' + temp_means + r' \\' + '\n'
        row_string += '         & ' + temp_stds + r' \\' + '\n'
        row_string += r'\hline' '\n'

    print(start_table + m_table + row_string + end_table)

In [13]:
make_table(MC_means, MC_ses)


    \begin{table}
    \centering
    \begin{tabular}{llll}
        & m = 1000 & m = 2000 & m = 3000 & m = 4000 & m = 5000 \\
\hline \hline
n = 250 & -0.0033223 & -0.003254 & -0.0032474 & -0.0032551 & -0.0032365 \\
         & (0.000003) & (0.000002) & (0.000001) & (0.000001) & (0.000001) \\
\hline
n = 500 & -0.0032002 & -0.0032108 & -0.0031933 & -0.0031867 & -0.0031922 \\
         & (0.000002) & (0.000001) & (0.000001) & (0.000001) & (0.000001) \\
\hline
n = 750 & -0.0031979 & -0.0031826 & -0.0031747 & -0.0031799 & -0.0031916 \\
         & (0.000002) & (0.000001) & (0.000001) & (0.000001) & (0.000001) \\
\hline

    \end{tabular}
    \end{table}
    
